# Turboshaft metamodel

In this notebook we will try to create a metamodel to estimate the performances of a turboshaft

In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

## Max power model

In [ ]:
import pandas as pd

THERMODYNAMIC_POWER_COLUMN_NAME = "Design Power (kW)"


def identify_design(df: pd.DataFrame):
    """
    We'll assume that the chance of two designs in the data having the same design power is
    minimal (though not theoretically impossible) so we'll identify designs by their power.
    """

    design_powers = df[THERMODYNAMIC_POWER_COLUMN_NAME].to_list()
    unique_design_powers = list(set(design_powers))

    return unique_design_powers

The next cell plots the behavior of the max power according to the OPR and ITT limit for different altitude and Mach number for one design, the behaviour seems quite monotonous so we'll try to approximate that behaviour with a polynomial regression and then define laws to find how the coefficient for the polynomials vary from one design to the other.

In [ ]:
import pathlib

import plotly.graph_objects as go

path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "max_power.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

designs = identify_design(existing_data)

# Show the max power curves for the first design
first_design_dataframe = existing_data.loc[
    existing_data[THERMODYNAMIC_POWER_COLUMN_NAME] == designs[0]
]

altitude_list = list(set(first_design_dataframe["Altitude (ft)"].to_list()))
mach_list = list(set(first_design_dataframe["Mach (-)"].to_list()))

max_power_opr = first_design_dataframe["Max Power OPR Limit (kW)"].to_list()
max_power_itt = first_design_dataframe["Max Power ITT Limit (kW)"].to_list()

fig = go.Figure()

altitude_list.sort()

for idx, alt in enumerate(altitude_list):

    dataframe_current_alt = first_design_dataframe.loc[
        first_design_dataframe["Altitude (ft)"] == alt
    ]

    scatter_current_alt_opr_limit = go.Scatter(
        x=dataframe_current_alt["Mach (-)"].to_list(),
        y=dataframe_current_alt["Max Power OPR Limit (kW)"].to_list(),
        mode="lines+markers",
        name="Max power OPR limit",
        legendgroup=str(alt),
        legendgrouptitle_text=str(alt),
    )
    fig.add_trace(scatter_current_alt_opr_limit)
    scatter_current_alt_itt_limit = go.Scatter(
        x=dataframe_current_alt["Mach (-)"].to_list(),
        y=dataframe_current_alt["Max Power ITT Limit (kW)"].to_list(),
        mode="lines+markers",
        name="Max power ITT limit",
        legendgroup=str(alt),
    )
    fig.add_trace(scatter_current_alt_itt_limit)

fig.update_layout(height=1000)
fig.show()

Let's first try on the first design

In [ ]:
import numpy as np

# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

from stdatm import Atmosphere

altitudes = np.array(first_design_dataframe["Altitude (ft)"].to_list())
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

pi1 = list(sigmas)
pi2 = first_design_dataframe["Mach (-)"].to_list()
pi3 = first_design_dataframe["Max Power OPR Limit (kW)"].to_list()
doe_opr_limit = np.c_[pi1, pi2, pi3]
doe_opr_limit = pd.DataFrame(doe_opr_limit, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_opr_limit.values,
    elected_pi0="pi3",
    order=2,
    log_space=False,
    ymax_axis=10,
    test_mode=True,
    plots=False,
)
average_rel_error = models["ave. e"][1][-1]
standard_deviation = models["sigma e"][1][-1]
print("average_rel_error", average_rel_error)
print("standard_deviation", standard_deviation)
_ = perform_regression(doe_opr_limit.values, models, chosen_model=6, no_plots=False)

Let's now do it for all designs for the OPR limit

In [ ]:
atm_0 = Atmosphere(0.0, altitude_in_feet=True)

for design in designs:

    current_design = existing_data.loc[existing_data[THERMODYNAMIC_POWER_COLUMN_NAME] == design]

    print("\nDesign Power (kW): " + str(current_design["Design Power (kW)"].to_list()[0]))

    altitudes = np.array(current_design["Altitude (ft)"].to_list())
    atm = Atmosphere(altitudes, altitude_in_feet=True)
    sigmas = atm.density / atm_0.density

    pi1 = list(sigmas)
    pi2 = current_design["Mach (-)"].to_list()
    pi3 = current_design["Max Power OPR Limit (kW)"].to_list()
    doe_opr_limit = np.c_[pi1, pi2, pi3]
    doe_opr_limit = pd.DataFrame(doe_opr_limit, columns=["pi1", "pi2", "pi3"])

    # Fit with 3rd order power-law model the obtained Pi DOE
    models = regression_models(
        doe_opr_limit.values,
        elected_pi0="pi3",
        order=2,
        log_space=False,
        ymax_axis=10,
        test_mode=True,
        plots=False,
    )
    _ = perform_regression(doe_opr_limit.values, models, chosen_model=6, no_plots=False)

And now for the ITT limit

In [ ]:
atm_0 = Atmosphere(0.0, altitude_in_feet=True)

for design in designs:

    current_design = existing_data.loc[existing_data[THERMODYNAMIC_POWER_COLUMN_NAME] == design]

    print("\nDesign Power (kW): " + str(current_design["Design Power (kW)"].to_list()[0]))

    altitudes = np.array(current_design["Altitude (ft)"].to_list())
    atm = Atmosphere(altitudes, altitude_in_feet=True)
    sigmas = atm.density / atm_0.density

    pi1 = list(sigmas)
    pi2 = current_design["Mach (-)"].to_list()
    pi3 = current_design["Max Power ITT Limit (kW)"].to_list()
    doe_opr_limit = np.c_[pi1, pi2, pi3]
    doe_opr_limit = pd.DataFrame(doe_opr_limit, columns=["pi1", "pi2", "pi3"])

    # Fit with 3rd order power-law model the obtained Pi DOE
    models = regression_models(
        doe_opr_limit.values,
        elected_pi0="pi3",
        order=2,
        log_space=False,
        ymax_axis=10,
        test_mode=True,
        plots=False,
    )
    _ = perform_regression(doe_opr_limit.values, models, chosen_model=6, no_plots=False)

In [ ]:
import plotly.express as px

result_file_path_max_power_coeff = data_folder_path / "max_power_coefficient.csv"

existing_data_coeff = pd.read_csv(result_file_path_max_power_coeff, index_col=None)

power_des = existing_data_coeff["Design Power (kW)"].tolist()
t41t_des = existing_data_coeff["Design T41t (degK)"].tolist()
opr_des = existing_data_coeff["Design OPR (-)"].tolist()
t45t_lim = existing_data_coeff["Limit ITT (degK)"].tolist()
opr_lim = existing_data_coeff["Limit OPR (-)"].tolist()

cst_term_opr = existing_data_coeff["Cst term OPR"].tolist()
s_term_opr = existing_data_coeff["s OPR"].tolist()
s_m_term_opr = existing_data_coeff["s * M OPR"].tolist()
s2_term_opr = existing_data_coeff["s2 OPR"].tolist()
m2_term_opr = existing_data_coeff["M2 OPR"].tolist()
m_term_opr = existing_data_coeff["M OPR"].tolist()

cst_term_itt = existing_data_coeff["Cst term ITT"].tolist()
s_term_itt = existing_data_coeff["s ITT"].tolist()
s_m_term_itt = existing_data_coeff["s * M ITT"].tolist()
s2_term_itt = existing_data_coeff["s2 ITT"].tolist()
m2_term_itt = existing_data_coeff["M2 ITT"].tolist()
m_term_itt = existing_data_coeff["M ITT"].tolist()

doe_cst_term_opr = np.c_[power_des, t41t_des, opr_des, opr_lim, cst_term_opr]
doe_cst_term_opr = pd.DataFrame(doe_cst_term_opr, columns=["pi1", "pi2", "pi3", "pi4", "pi5"])

fig = px.scatter_matrix(doe_cst_term_opr, width=1200, height=800)
fig.show()

In [ ]:
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
t0 = atm_0.temperature
print(t0)

# doe_cst_term_opr = np.c_[np.array(power_des), np.array(t41t_des) / t0, opr_lim, np.array(s_term_opr)]
# doe_cst_term_opr = pd.DataFrame(doe_cst_term_opr, columns=["pi1", "pi2", "pi3", "pi4"])

# models = regression_models(
#     doe_cst_term_opr.values, elected_pi0="pi4", order=2, log_space=False, ymax_axis=10, test_mode=True, plots=True
# )
# _ = perform_regression(doe_cst_term_opr.values, models, chosen_model=6, no_plots=False)

In [ ]:
B = np.log(np.array(s_term_opr))
A = np.c_[
    np.ones_like(power_des),
    np.log(np.array(power_des)),
    np.log(np.array(t41t_des)),
    np.log(np.array(opr_lim)),
]
x = np.linalg.lstsq(A, B, rcond=None)
k, a_exp, b_exp, c_exp = x[0]
print(x[1])

print(
    list(
        np.round(
            np.exp(k)
            * np.array(power_des) ** a_exp
            * np.array(t41t_des) ** b_exp
            * np.array(opr_lim) ** c_exp,
            5,
        )
    )
)
print(s_term_opr)

# Brute Force

For the OPR limit

In [ ]:
path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "max_power.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

pi1 = list(sigmas)
pi2 = existing_data["Mach (-)"].to_list()
pi3 = existing_data["Design Power (kW)"].to_list()
pi4 = existing_data["Design T41t (degK)"].to_list()
pi5 = existing_data["Design OPR (-)"].to_list()
pi6 = existing_data["Limit OPR (-)"].to_list()
pi7 = existing_data["Max Power OPR Limit (kW)"].to_list()
doe_opr_limit = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_opr_limit = pd.DataFrame(
    doe_opr_limit, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_opr_limit.values,
    elected_pi0="pi7",
    order=3,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
_ = perform_regression(doe_opr_limit.values, models, chosen_model=15, no_plots=False)

For the ITT limit

In [ ]:
path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "max_power.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

pi1 = list(sigmas)
pi2 = existing_data["Mach (-)"].to_list()
pi3 = existing_data["Design Power (kW)"].to_list()
pi4 = existing_data["Design T41t (degK)"].to_list()
pi5 = existing_data["Design OPR (-)"].to_list()
pi6 = existing_data["Limit ITT (degK)"].to_list()
pi7 = existing_data["Max Power ITT Limit (kW)"].to_list()
doe_itt_limit = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_itt_limit = pd.DataFrame(
    doe_itt_limit, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_itt_limit.values,
    elected_pi0="pi7",
    order=3,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
_ = perform_regression(doe_itt_limit.values, models, chosen_model=15, no_plots=False)

# Now for the fuel consumed

In [ ]:
path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "fuel_consumed.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

index_to_retain = np.random.randint(0, len(altitudes), size=750)

pi1 = sigmas[index_to_retain]
pi2 = existing_data["Mach (-)"].to_numpy()[index_to_retain]
pi3 = existing_data["Design Power (kW)"].to_numpy()[index_to_retain]
pi4 = existing_data["Design T41t (degK)"].to_numpy()[index_to_retain]
pi5 = existing_data["Design OPR (-)"].to_numpy()[index_to_retain]
pi6 = existing_data["Shaft Power (kW)"].to_numpy()[index_to_retain]
pi7 = existing_data["Fuel mass flow (kg/h)"].to_numpy()[index_to_retain]
doe_fuel_consumed_limit = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_fuel_consumed_limit = pd.DataFrame(
    doe_fuel_consumed_limit, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_fuel_consumed_limit.values,
    elected_pi0="pi7",
    order=3,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
_ = perform_regression(doe_fuel_consumed_limit.values, models, chosen_model=18, no_plots=False)

In [ ]:
print(min(existing_data["Fuel mass flow (kg/h)"].to_numpy()))
print(max(existing_data["Fuel mass flow (kg/h)"].to_numpy()))

Now for the exhaust thrust

In [ ]:
import pathlib
import numpy as np
import pandas as pd
from stdatm import Atmosphere

path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "fuel_consumed.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

index_to_retain = np.random.randint(0, len(altitudes), size=750)
exhaust_thrust = existing_data["Exhaust Thrust (N)"].to_numpy()[index_to_retain]
# only taking positive value of exhaust thrust
new_index = np.where(exhaust_thrust > 0)

pi1 = sigmas[index_to_retain][new_index]
pi2 = existing_data["Mach (-)"].to_numpy()[index_to_retain][new_index]
pi3 = existing_data["Design Power (kW)"].to_numpy()[index_to_retain][new_index]
pi4 = existing_data["Design T41t (degK)"].to_numpy()[index_to_retain][new_index]
pi5 = existing_data["Design OPR (-)"].to_numpy()[index_to_retain][new_index]
pi6 = existing_data["Shaft Power (kW)"].to_numpy()[index_to_retain][new_index]
pi7 = existing_data["Exhaust Thrust (N)"].to_numpy()[index_to_retain][new_index]
doe_exhaust_thrust = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_exhaust_thrust = pd.DataFrame(
    doe_exhaust_thrust, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
# models = regression_models(
#     doe_exhaust_thrust.values, elected_pi0="pi7", order=4, log_space=True, ymax_axis=10, test_mode=True, plots=True
# )

In [ ]:
# _ = perform_regression(doe_exhaust_thrust.values, models, chosen_model=30, no_plots=False)

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(doe_exhaust_thrust, width=1200, height=800)
fig.show()

# Exhaust thrust but differently

In [ ]:
import pathlib
import numpy as np
import pandas as pd
from stdatm import Atmosphere

# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "fuel_consumed_complemented.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

index_to_retain = np.random.randint(0, len(altitudes), size=750)

pi1 = sigmas[index_to_retain]
pi2 = existing_data["Mach (-)"].to_numpy()[index_to_retain]
pi3 = existing_data["Design Power (kW)"].to_numpy()[index_to_retain]
pi4 = existing_data["Design T41t (degK)"].to_numpy()[index_to_retain]
pi5 = existing_data["Design OPR (-)"].to_numpy()[index_to_retain]
pi6 = existing_data["Shaft Power (kW)"].to_numpy()[index_to_retain]
pi7 = existing_data["Exhaust velocity (m/s)"].to_numpy()[index_to_retain]

doe_exhaust_velocity = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_exhaust_velocity = pd.DataFrame(
    doe_exhaust_velocity, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_exhaust_velocity.values,
    elected_pi0="pi7",
    order=3,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
_ = perform_regression(doe_exhaust_velocity.values, models, chosen_model=15, no_plots=False)

In [ ]:
import pathlib
import numpy as np
import pandas as pd
from stdatm import Atmosphere

# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "fuel_consumed_complemented.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

index_to_retain = np.random.randint(0, len(altitudes), size=750)

pi1 = sigmas[index_to_retain]
pi2 = existing_data["Mach (-)"].to_numpy()[index_to_retain]
pi3 = existing_data["Design Power (kW)"].to_numpy()[index_to_retain]
pi4 = existing_data["Design T41t (degK)"].to_numpy()[index_to_retain]
pi5 = existing_data["Design OPR (-)"].to_numpy()[index_to_retain]
pi6 = existing_data["Shaft Power (kW)"].to_numpy()[index_to_retain]
pi7 = existing_data["Exhaust mass flow (kg/s)"].to_numpy()[index_to_retain]

doe_exhaust_mass_flow = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_exhaust_mass_flow = pd.DataFrame(
    doe_exhaust_mass_flow, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_exhaust_mass_flow.values,
    elected_pi0="pi7",
    order=2,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
_ = perform_regression(doe_exhaust_mass_flow.values, models, chosen_model=10, no_plots=False)

# Max power OPT and ITT reversed

I turns out, the original orientation of the model wasn't what we needed. What we need is a way to derive, from the power seen during the mission, what the minimum thermodynamic design power would be that allow to reach that power, hence why we'll reverse the model.

In [ ]:
import pathlib
import numpy as np
import pandas as pd
from stdatm import Atmosphere

# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "max_power_v2.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

pi1 = list(sigmas)
pi2 = existing_data["Mach (-)"].to_list()
pi3 = existing_data["Max Power OPR Limit (kW)"].to_numpy() / 1000.0
pi4 = existing_data["Design T41t (degK)"].to_numpy() / atm_0.temperature
pi5 = existing_data["Design OPR (-)"].to_list()
pi6 = existing_data["Limit OPR (-)"].to_list()
pi7 = existing_data["Design Power (kW)"].to_numpy() / 1000.0
doe_opr_limit = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_opr_limit = pd.DataFrame(
    doe_opr_limit, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_opr_limit.values,
    elected_pi0="pi7",
    order=3,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
surrogate = perform_regression(doe_opr_limit.values, models, chosen_model=21, no_plots=False)

In [ ]:
import pathlib
import numpy as np
import pandas as pd
from stdatm import Atmosphere
import plotly.express as px

# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

path_to_current_file = pathlib.Path().cwd()
data_folder_path = path_to_current_file / "data"
result_file_path_max_power = data_folder_path / "max_power_v2.csv"

existing_data = pd.read_csv(result_file_path_max_power, index_col=0)

altitudes = existing_data["Altitude (ft)"].to_numpy()
machs = existing_data["Mach (-)"].to_numpy()
atm = Atmosphere(altitudes, altitude_in_feet=True)
atm_0 = Atmosphere(0.0, altitude_in_feet=True)
sigmas = atm.density / atm_0.density

pi1 = list(sigmas)
pi2 = existing_data["Mach (-)"].to_list()
pi3 = existing_data["Max Power ITT Limit (kW)"].to_numpy() / 1000.0
pi4 = existing_data["Design T41t (degK)"].to_numpy() / atm_0.temperature
pi5 = existing_data["Design OPR (-)"].to_list()
pi6 = existing_data["Limit ITT (degK)"].to_numpy() / atm_0.temperature
pi7 = existing_data["Design Power (kW)"].to_numpy() / 1000.0
doe_itt_limit = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7]
doe_itt_limit = pd.DataFrame(
    doe_itt_limit, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7"]
)

fig = px.scatter_matrix(doe_itt_limit, width=1200, height=800)
fig.show()

In [ ]:
# Fit with 3rd order power-law model the obtained Pi DOE
models = regression_models(
    doe_itt_limit.values,
    elected_pi0="pi7",
    order=3,
    log_space=True,
    ymax_axis=10,
    test_mode=True,
    plots=True,
)

In [ ]:
surrogate = perform_regression(doe_itt_limit.values, models, chosen_model=21, no_plots=False)